In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
url = 'https://warcraft.wiki.gg/wiki/'

In [39]:
def get_all_subzones(url_zone):  
   
    response = requests.get(url+url_zone)
    page = BeautifulSoup(response.text, 'html.parser')
    tag = page.html.body
    
    subregions_tag = next((x.parent for x in tag.find_all(True) if 'id' in x.attrs and x.attrs['id'] == "Maps_and_subregions"), None)
    
    if not subregions_tag:
        return []
    
    table = next(x for x in subregions_tag.next_siblings if x.name == 'table')
    
    subregions = []
    for item in table.find_all(True):
        if item.string and "Undisplayed" in item.string:
            break
        if item.string and item.name=='a' and item.parent.name=='li':
            subregions.append(item.string)
    
    return subregions

zone = 'Dun_Morogh'
zone2 = 'Elwynn_Forest'
#get_all_subzones(zone2)

In [69]:
def get_all_zones(url_continent):
    
    response = requests.get(url+url_continent)
    page = BeautifulSoup(response.text, 'html.parser')
    table = next(x for x in page.find_all(True) if x.name=='table' and 'class' in x.attrs and 'navbox' in x.attrs['class'])
            
    result = [item.attrs['href'][6:] 
              for item in table.descendants
              if item.name == 'a' 
              and item.parent.name == 'li' 
              and not 'Classic' in item.attrs['href']]
    
    return result

count = 0
for zone in get_all_zones('Eastern_Kingdoms'):
    zones = get_all_subzones(zone)
    count += len(zones)
    
count
    
    

993